# Final Verification: Side-by-Side Maps of Yield and Temperature

**Goal:** To visually inspect our final, cleaned dataset by plotting the maize yield and the corresponding growing season temperature for our Northern Italy region for every year from 1981 to 2016.

In [ ]:
# Final Verification (Corrected): Side-by-Side Maps of Yield and Temperature
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# --- 1. Load our final, clean dataset ---
file_path = '../data/analysis_ready/n_italy_maize_gridcell_temp_data.csv'
final_df = pd.read_csv(file_path)
print(f"Loaded the final dataset with {len(final_df)} rows.")

# --- 2. VERIFICATION 1: Count rows per year (as you suggested) ---
print("\n--- Verifying number of grid cells per year ---")
rows_per_year = final_df.groupby('year').size()
print(rows_per_year.to_string())

# --- 3. Clean the Data (as you suggested) ---
# Exclude 1981 because it has very sparse data
df_cleaned = final_df[final_df['year'] != 1981].copy()
print(f"\nRemoved 1981 data. The dataset now has {len(df_cleaned)} rows.")

# --- 4. Convert the clean DataFrame back into a gridded xarray Dataset ---
# This is the correct way to reconstruct the data cubes for plotting.
ds_final = df_cleaned.set_index(['year', 'lat', 'lon']).to_xarray()

# --- 5. Determine consistent color scales for plotting ---
yield_vmin = ds_final['maize_yield'].min().item()
yield_vmax = ds_final['maize_yield'].max().item()
temp_vmin = ds_final['temperature'].min().item()
temp_vmax = ds_final['temperature'].max().item()

# --- 6. Loop through each year and create a pair of maps ---
# We loop through the unique years present in our cleaned dataset
for year in ds_final.year.values:
    print(f"\n--- Generating maps for {year} ---")
    
    data_for_year = ds_final.sel(year=year)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), subplot_kw={'projection': ccrs.PlateCarree()})
    
    # --- Plot 1: Maize Yield ---
    ax1.set_extent([5, 19, 42, 48], crs=ccrs.PlateCarree())
    ax1.add_feature(cfeature.LAND, facecolor='#f0f0f0')
    ax1.coastlines()
    ax1.add_feature(cfeature.BORDERS, linestyle=':')
    data_for_year['maize_yield'].plot(
        ax=ax1, transform=ccrs.PlateCarree(), cmap='viridis',
        vmin=yield_vmin, vmax=yield_vmax,
        cbar_kwargs={'label': 'Maize Yield (t/ha)'}
    )
    ax1.set_title(f'Maize Yield in {year}')

    # --- Plot 2: Temperature ---
    ax2.set_extent([5, 19, 42, 48], crs=ccrs.PlateCarree())
    ax2.add_feature(cfeature.LAND, facecolor='#f0f0f0')
    ax2.coastlines()
    ax2.add_feature(cfeature.BORDERS, linestyle=':')
    data_for_year['temperature'].plot(
        ax=ax2, transform=ccrs.PlateCarree(), cmap='inferno',
        vmin=temp_vmin, vmax=temp_vmax,
        cbar_kwargs={'label': 'Avg. Growing Season Temp (°C)'}
    )
    ax2.set_title(f'Temperature in {year}')
    
    plt.show()

print("\n--- All verification maps generated. ---")